Inspiration 🥷
https://github.com/ZA3karia/PDF2TEXT/blob/master/ebook_to_text.ipynb

In [1]:
!pip install ebooklib
!pip install BeautifulSoup4
!pip install openai

In [2]:
# conda env nlp
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

In [3]:
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [4]:
blacklist = [
	'[document]',
	'noscript',
	'header',
	'html',
	'meta',
	'head', 
	'input',
	'script',
	# there may be more elements you don't want, such as "style", etc.
]
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output


In [5]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [6]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [7]:
out = epub2text('books/About Looking (Vintage International) - John Berger.epub')

/opt/homebrew/Caskroom/miniforge/base/envs/nlp/lib/python3.9/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [8]:
total_cost = 0
for i,c in enumerate(out):
    # Only transcribe chapters with more than 5000 characters
    if len(c) > 5000:
        # price is 0.015$ per 1000 characters
        print(i, len(c), c[:20].replace('\n', ''), f'cost {len(c)/1000*0.015:.2f}$')
        total_cost += len(c)/1000*0.015
print(f'total cost: {total_cost:.2f}$')

10 40413 WHY LOOK AT ANIMALS? cost 0.61$
12 12291 THE SUIT AND THE PHO cost 0.18$
13 6897 PHOTOGRAPHS OF AGONY cost 0.10$
14 9886 PAUL STRAND    Ther cost 0.15$
15 20453 USES OF PHOTOGRAPHY  cost 0.31$
17 9202 THE PRIMITIVE AND TH cost 0.14$
18 16747 MILLET AND THE PEASA cost 0.25$
19 13455 SEKER AHMET AND THE  cost 0.20$
20 14747 LOWRY AND THE INDUST cost 0.22$
21 11447 RALPH FASANELLA AND  cost 0.17$
22 13039 LA TOUR AND HUMANISM cost 0.20$
23 12343 FRANCIS BACON AND WA cost 0.19$
24 12042 AN ARTICLE OF FAITH  cost 0.18$
25 13031 BETWEEN TWO COLMARS  cost 0.20$
26 15040 COURBET AND THE JURA cost 0.23$
27 12705 TURNER AND THE BARBE cost 0.19$
28 10279 ROUAULT AND THE SUBU cost 0.15$
29 11689 MAGRITTE AND THE IMP cost 0.18$
30 15248 HALS AND BANKRUPTCY  cost 0.23$
31 8692 GIACOMETTI    The w cost 0.13$
32 14292 RODIN AND SEXUAL DOM cost 0.21$
33 10014 ROMAINE LORQUET     cost 0.15$
34 9866 FIELD    “Life is n cost 0.15$
total cost: 4.71$


In [10]:
from openai import OpenAI

import os
os.environ["OPENAI_API_KEY"] = "<your key>"
client = OpenAI()

index = 0
for i,c in enumerate(out):
    # Only transcribe chapters with more than 5000 characters
    if len(c) > 5000:
      print(i, c[:20].replace('\n', ''))
      chunk = ''
      chunks = []
      for t in c.split('\n'):
          if len(chunk) + len(t) < 4000:
              chunk += t + '\n'
          else:
              chunks.append(chunk)
              chunk = t + '\n'
      for i,payload in enumerate(chunks):
          response = client.audio.speech.create(
              model="tts-1",
              voice="onyx",
              input=payload,
            )  
          response.stream_to_file(f'{index:03} {c[:20]}_{i}.mp3')
          index += 1

10 WHY LOOK AT ANIMALS?
12 THE SUIT AND THE PHO
13 PHOTOGRAPHS OF AGONY
14 PAUL STRAND    Ther
15 USES OF PHOTOGRAPHY 
17 THE PRIMITIVE AND TH
18 MILLET AND THE PEASA
19 SEKER AHMET AND THE 
20 LOWRY AND THE INDUST
21 RALPH FASANELLA AND 
22 LA TOUR AND HUMANISM
23 FRANCIS BACON AND WA
24 AN ARTICLE OF FAITH 
25 BETWEEN TWO COLMARS 
26 COURBET AND THE JURA
27 TURNER AND THE BARBE
28 ROUAULT AND THE SUBU
29 MAGRITTE AND THE IMP
30 HALS AND BANKRUPTCY 
31 GIACOMETTI    The w
32 RODIN AND SEXUAL DOM
33 ROMAINE LORQUET    
34 FIELD    “Life is n
